In [ ]:
import cv2
import mediapipe as mp
import pyautogui
from pynput.mouse import Button, Controller
import time

import util

In [ ]:
mouse = Controller()

width, hegiht = pyautogui.size()
sensitivity = 1.5

mphand = mp.solutions.hands
hand = mphand.Hands(
    static_image_mode = False,
    model_complexity = 1,
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.7,
    max_num_hands = 1
)

In [ ]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        
        return hand_landmarks.landmark[mphand.HandLandmark.INDEX_FINGER_TIP]
    
    return None

In [ ]:
def move_cursor(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * width * sensitivity)
        y = int(index_finger_tip.y * hegiht * sensitivity)
        pyautogui.moveTo(x, y)

In [ ]:
def is_left_click(landmarks_list, thumb_index_dist):
    return (
        thumb_index_dist > 50 and 
        util.angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and 
        util.angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) > 90
            )

def is_right_click(landmarks_list, thumb_index_dist):
    return (
        thumb_index_dist > 50 and 
        util.angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) > 90 and 
        util.angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 50
            )
            
def is_double_click(landmarks_list, thumb_index_dist):
    return(
        thumb_index_dist > 50 
        and util.angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50
        and util.angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 50
           )

In [ ]:
def detect_gestures(frame, landmarks_list, processed):
    if len(landmarks_list) >= 21:
        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = util.distance((landmarks_list[4], landmarks_list[5]))

        if thumb_index_dist < 50 and util.angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) > 90:
            move_cursor(index_finger_tip)
            cv2.putText(frame, "Move Cursor", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 0), 7, cv2.LINE_AA)
            cv2.putText(frame, "Move Cursor", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 3, cv2.LINE_AA)

        elif is_left_click(landmarks_list, thumb_index_dist):
            mouse.press(Button.left)
            mouse.release(Button.left)
            time.sleep(0.1)
            cv2.putText(frame, "Left Click", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 0), 7, cv2.LINE_AA)
            cv2.putText(frame, "Left Click", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 3, cv2.LINE_AA)

        elif is_right_click(landmarks_list, thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            time.sleep(0.1)
            cv2.putText(frame, "Right Click", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 0), 7, cv2.LINE_AA)
            cv2.putText(frame, "Right Click", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 3, cv2.LINE_AA)

        elif is_double_click(landmarks_list, thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame, "Double Click", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 0), 7, cv2.LINE_AA)
            cv2.putText(frame, "Doubel Click", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 3, cv2.LINE_AA)    

In [ ]:
cap = cv2.VideoCapture(0)
draw = mp.solutions.drawing_utils

try:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        frame = cv2.flip(frame, 1)
        frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        processed = hand.process(frameRGB)

        landmarks_list = []

        if processed.multi_hand_landmarks:
            hand_landmarks = processed.multi_hand_landmarks[0]
            draw.draw_landmarks(frame, hand_landmarks, mphand.HAND_CONNECTIONS)

            for landmark in hand_landmarks.landmark:
                landmarks_list.append((landmark.x, landmark.y))

        detect_gestures(frame, landmarks_list, processed)


        print(landmarks_list)


        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 

finally:
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)